In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[5],3c
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,300,300


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.253565                    1  0.099301
0.000750        2  0.253741                    2  0.103173
0.001052        3  0.264521                    3  0.112968
0.001476        4  0.274320                    4  0.122761
0.002070        5  0.282196                    5  0.132450
0.002904        6  0.287747                    6  0.141552
0.004074        7  0.290000                    7  0.150029
0.005714        8  0.288125                    8  0.157213
0.008015        9  0.280969                    9  0.162391
0.011243       10  0.267291                   10  0.164822
0.015771       11  0.245779                   11  0.162950
0.022122       12  0.214917                   12  0.155135
0.031031       13  0.172816                   13  0.138547
0.043528       14  0.120400                   14  0.113662
0.061057       15  0.101435                   15  0.116020
0.085645       16  0.139053                   16  0.167004
0.120136       17  0.189440                   17  0.227964
0.168516       18  0.226890                   18  0.263829
0.236378       19  0.258186                   19  0.271122
0.331549       20  0.295526                   20  0.271045
0.465100       21  0.361773                   21  0.311906
0.652400       22  0.476333                   22  0.432367
0.915100       23  0.483616                   23  0.464961
1.283650       24  0.403821                   24  0.407642
1.800600       25  0.338784                   25  0.350213
2.525700       26  0.268299                   26  0.279798
3.542800       27  0.213223                   27  0.228593
4.969550       28  0.171281                   28  0.191234
6.970850       29  0.136919                   29  0.158966
9.778100       30  0.107628                   30  0.124126
13.715850      31  0.082373                   31  0.090047
19.239350      32  0.060832                   32  0.059925
26.987250      33  0.043207                   33  0.036588
37.855300      34  0.030233                   34  0.023142
53.100050      35  0.012333                   35  0.008623
73.887500      36 -0.008562                   36 -0.010218
97.662500      37 -0.021921                   37 -0.023888
121.437500     38 -0.020843                   38 -0.023393
145.212500     39 -0.010451                   39 -0.013725
168.987500     40 -0.004142                   40 -0.007542
192.762500     41  0.002087                   41 -0.000887
216.537500     42  0.008626                   42  0.005790
240.312500     43  0.015568                   43  0.012465
264.087500     44  0.023001                   44  0.019117
287.862500     45  0.031047                   45  0.026036
311.637500     46  0.039816                   46  0.033520
335.412500     47  0.049340                   47  0.041788
359.187500     48  0.059396                   48  0.051137
382.962500     49  0.069099                   49  0.060656
406.737500     50  0.078963                   50  0.070654
430.512500     51  0.088914                   51  0.080908
454.287500     52  0.098894                   52  0.091282
478.062500     53  0.108865                   53  0.101988
501.837500     54  0.118759                   54  0.112717
525.612500     55  0.128643                   55  0.123980
549.387500     56  0.138468                   56  0.135645
573.162500     57  0.148299                   57  0.147853
596.937500     58  0.158274                   58  0.160049
620.712500     59  0.168471                   59  0.172087
644.487500     60  0.179157                   60  0.184068
668.262500     61  0.190096                   61  0.196015
692.037500     62  0.199786                   62  0.206779
715.812500     63  0.205715                   63  0.213962
739.587500     64  0.212600                   64  0.222866
763.362500     65  0.213038                   65  0.225634
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -26.007355   0.000000 -26.007355 -25.692252  1.846029e-09   
0.000624    2     -26.007370   0.000034 -26.007336 -25.692268  2.415998e-05   
0.000876    3     -26.007376   0.000048 -26.007328 -25.692275  3.399045e-05   
0.001229    4     -26.007385   0.000068 -26.007317 -25.692285  4.810845e-05   
0.001723    5     -26.007397   0.000096 -26.007301 -25.692298  6.837981e-05   
0.002417    6     -26.007413   0.000135 -26.007278 -25.692316  9.745446e-05   
0.003391    7     -26.007436   0.000191 -26.007245 -25.692341  1.390924e-04   
0.004757    8     -26.007467   0.000269 -26.007198 -25.692376  1.986204e-04   
0.006672    9     -26.007511   0.000378 -26.007133 -25.692426  2.834847e-04   
0.009359    10    -26.007571   0.000528 -26.007043 -25.692494  4.040047e-04   
0.013128    11    -26.007656   0.000732 -26.006924 -25.692591  5.743466e-04   
0.018415    12    -26.007777   0.001007 -26.006770 -25.692728  8.134548e-04   
0.025830    13    -26.007949   0.001368 -26.006581 -25.692924  1.145670e-03   
0.036232    14    -26.008203   0.001834 -26.006368 -25.693208  1.600613e-03   
0.050823    15    -26.008587   0.002427 -26.006160 -25.693627  2.216111e-03   
0.071291    16    -26.009159   0.003245 -26.005914 -25.694233  3.102874e-03   
0.100000    17    -26.009951   0.004510 -26.005441 -25.695058  4.496008e-03   
0.140271    18    -26.010993   0.006455 -26.004538 -25.696128  6.653192e-03   
0.196760    19    -26.012318   0.009298 -26.003019 -25.697465  9.756336e-03   
0.275997    20    -26.013922   0.013326 -26.000596 -25.699055  1.389179e-02   
0.387100    21    -26.015682   0.018975 -25.996707 -25.700770  1.917353e-02   
0.543100    22    -26.017045   0.027024 -25.990021 -25.702060  2.622799e-02   
0.761700    23    -26.016161   0.038474 -25.977687 -25.700988  3.635360e-02   
1.068500    24    -26.012293   0.052182 -25.960111 -25.696774  4.903937e-02   
1.498800    25    -26.006452   0.066925 -25.939527 -25.690418  6.346384e-02   
2.102400    26    -25.998775   0.083471 -25.915304 -25.681988  8.007632e-02   
2.949000    27    -25.990002   0.101604 -25.888397 -25.672162  9.831340e-02   
4.136600    28    -25.980343   0.121942 -25.858401 -25.660982  1.192952e-01   
5.802500    29    -25.969744   0.145143 -25.824601 -25.648254  1.443089e-01   
8.139200    30    -25.958264   0.171562 -25.786702 -25.633771  1.738319e-01   
11.417000   31    -25.946157   0.201245 -25.744912 -25.617734  2.059948e-01   
16.014700   32    -25.933976   0.233927 -25.700049 -25.600715  2.380237e-01   
22.464000   33    -25.922707   0.269131 -25.653576 -25.584086  2.671800e-01   
31.510500   34    -25.913668   0.306394 -25.607274 -25.569785  2.920918e-01   
44.200100   35    -25.906755   0.344927 -25.561828 -25.557947  3.150446e-01   
62.000000   36    -25.911626   0.375801 -25.535825 -25.560582  3.358639e-01   
85.775000   37    -25.952730   0.392792 -25.559937 -25.607068  3.535703e-01   
109.550000  38    -26.031550   0.409876 -25.621675 -25.697404  3.766235e-01   
133.325000  39    -26.142025   0.461651 -25.680374 -25.822339  4.356692e-01   
157.100000  40    -26.271357   0.561549 -25.709808 -25.966534  5.412063e-01   
180.875000  41    -26.417654   0.696180 -25.721474 -26.127301  6.807308e-01   
204.650000  42    -26.579299   0.863702 -25.715597 -26.303342  8.542730e-01   
228.425000  43    -26.755006   1.063704 -25.691302 -26.494536  1.061775e+00   
252.200000  44    -26.943519   1.296061 -25.647457 -26.699098  1.301446e+00   
275.975000  45    -27.143906   1.561226 -25.582679 -26.915618  1.571810e+00   
299.750000  46    -27.355483   1.860242 -25.495240 -27.143166  1.872692e+00   
323.525000  47    -27.577778   2.194673 -25.383105 -27.380917  2.204857e+00   
347.300000  48    -27.810506   2.566359 -25.244147 -27.628328  2.569969e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')